# Satellite Analysis Report

**Template for Consulting Engagements**

This notebook generates a comprehensive satellite-based analysis report for a client location. Modify the configuration cell below and run all cells to generate the deliverable.

---

## 1. Client Configuration

**MODIFY THIS CELL** for each new project. All subsequent cells use these variables.

In [ ]:
# ╔════════════════════════════════════════════════════════════════════╗
# ║                    CLIENT CONFIGURATION                            ║
# ║         Modify these values for each consulting project            ║
# ╚════════════════════════════════════════════════════════════════════╝

# Client Information
CLIENT_NAME = "Example Client"  # Client or project name
REPORT_DATE = "2026-02-05"      # Report generation date

# Location (WGS84 coordinates)
LATITUDE = 51.41   # Degrees North
LONGITUDE = 21.97  # Degrees East
LOCATION_NAME = "Lublin Region, Poland"  # Human-readable name

# Analysis Period
ANALYSIS_DAYS = 30  # Number of days to analyze (default: 30)
COMPARISON_DAYS = 60  # For change detection: baseline period ends this many days ago

# Output Configuration
OUTPUT_DIR = "./output"  # Directory for exported files
EXPORT_PNG = True        # Export PNG visualizations
EXPORT_CSV = True        # Export CSV data files

# ════════════════════════════════════════════════════════════════════

print(f"Configuration loaded for: {CLIENT_NAME}")
print(f"Location: {LOCATION_NAME} ({LATITUDE}°N, {LONGITUDE}°E)")
print(f"Analysis period: last {ANALYSIS_DAYS} days")

## 2. Setup & Initialization

In [ ]:
import os
from datetime import datetime
from pathlib import Path

import satellitehub as sh

# Create output directory
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Create location reference
location = sh.location(lat=LATITUDE, lon=LONGITUDE)

print(f"SatelliteHub version: {sh.__version__}")
print(f"Location initialized: {location}")
print(f"UTM Zone: {location.utm_zone} (EPSG:{location.utm_epsg})")
print(f"Output directory: {output_path.absolute()}")

In [ ]:
# Check available analysis methods
print("Available analysis methods:")
print("-" * 50)
for method in location.available_methods():
    print(method)

## 3. Data Acquisition

Query satellite and weather data for the analysis period.

In [ ]:
# Check data availability
print(f"Checking data availability for last {ANALYSIS_DAYS} days...")
data_summary = location.available_data(last_days=ANALYSIS_DAYS)

print(f"\nTotal satellite passes: {data_summary['total_passes']}")
print(f"Date range: {data_summary['date_range'][0]} to {data_summary['date_range'][1]}")

for provider, passes in data_summary['providers'].items():
    print(f"  {provider}: {len(passes)} passes")

if data_summary['warnings']:
    print("\nWarnings:")
    for w in data_summary['warnings']:
        print(f"  ⚠ {w}")

## 4. Vegetation Health Analysis

Compute NDVI-based vegetation health index from Sentinel-2 imagery.

In [ ]:
# Run vegetation health analysis
print(f"Analyzing vegetation health for {LOCATION_NAME}...")
print(f"Period: last {ANALYSIS_DAYS} days\n")

veg_result = location.vegetation_health(last_days=ANALYSIS_DAYS)
print(veg_result)

In [ ]:
# Vegetation metrics summary
print("\n" + "="*50)
print("VEGETATION HEALTH SUMMARY")
print("="*50)
print(f"Location: {LOCATION_NAME}")
print(f"Coordinates: {LATITUDE}°N, {LONGITUDE}°E")
print(f"Analysis Period: {ANALYSIS_DAYS} days")
print("-"*50)
print(f"Mean NDVI: {veg_result.mean_ndvi:.3f}")
print(f"NDVI Std Dev: {veg_result.ndvi_std:.3f}")
print(f"Confidence: {veg_result.confidence:.1%}")
print(f"Cloud-free observations: {veg_result.cloud_free_count}/{veg_result.observation_count}")

if veg_result.trend is not None:
    trend_dir = "improving" if veg_result.trend > 0 else "declining" if veg_result.trend < 0 else "stable"
    print(f"Trend: {trend_dir} ({veg_result.trend:+.3f}/month)")

print("="*50)

## 5. Vegetation Change Detection

Compare current vegetation health to the baseline period.

In [ ]:
# Run change detection
print(f"Detecting vegetation changes...")
print(f"Baseline: {COMPARISON_DAYS}-{ANALYSIS_DAYS} days ago")
print(f"Current: last {ANALYSIS_DAYS} days\n")

change_result = location.vegetation_change(
    period_1_days=COMPARISON_DAYS,
    period_2_days=ANALYSIS_DAYS
)
print(change_result)

In [ ]:
# Change detection summary
print("\n" + "="*50)
print("VEGETATION CHANGE SUMMARY")
print("="*50)
print(f"Baseline NDVI: {change_result.period_1_ndvi:.3f}")
print(f"Current NDVI: {change_result.period_2_ndvi:.3f}")
print(f"Change: {change_result.delta:+.3f}")
print(f"Direction: {change_result.direction.upper()}")
print(f"Confidence: {change_result.confidence:.1%}")
print("="*50)

## 6. Weather Context

Retrieve temperature and precipitation data to contextualize vegetation health.

In [ ]:
# Get weather data
print(f"Retrieving weather data for {LOCATION_NAME}...\n")

weather_result = location.weather(last_days=ANALYSIS_DAYS)
print(weather_result)

In [ ]:
# Weather summary
if weather_result.confidence > 0:
    print("\n" + "="*50)
    print("WEATHER SUMMARY")
    print("="*50)
    print(f"Mean Temperature: {weather_result.mean_temperature:.1f}°C")
    print(f"Temperature Range: {weather_result.temperature_min:.1f}°C to {weather_result.temperature_max:.1f}°C")
    print(f"Total Precipitation: {weather_result.total_precipitation:.1f} mm")
    print(f"Data Source: {weather_result.data_source}")
    print(f"Observations: {weather_result.observation_count} days")
    print("="*50)
else:
    print("\n⚠ Weather data not available (CDS credentials may not be configured)")

## 7. Results Summary

Combined analysis results for the client report.

In [ ]:
# Executive Summary
print("\n" + "#"*60)
print("#" + " "*20 + "EXECUTIVE SUMMARY" + " "*21 + "#")
print("#"*60)
print(f"\nClient: {CLIENT_NAME}")
print(f"Report Date: {REPORT_DATE}")
print(f"Location: {LOCATION_NAME}")
print(f"Coordinates: {LATITUDE}°N, {LONGITUDE}°E")
print(f"Analysis Period: {ANALYSIS_DAYS} days")

print("\n--- Vegetation Health ---")
print(f"Current NDVI: {veg_result.mean_ndvi:.2f} (confidence: {veg_result.confidence:.0%})")

# Interpret NDVI
if veg_result.mean_ndvi >= 0.6:
    veg_status = "HEALTHY - Dense, vigorous vegetation"
elif veg_result.mean_ndvi >= 0.3:
    veg_status = "MODERATE - Normal vegetation cover"
elif veg_result.mean_ndvi >= 0.1:
    veg_status = "SPARSE - Stressed or sparse vegetation"
else:
    veg_status = "MINIMAL - Bare soil or water"
print(f"Status: {veg_status}")

print("\n--- Vegetation Change ---")
print(f"Change: {change_result.delta:+.2f} ({change_result.direction})")

if weather_result.confidence > 0:
    print("\n--- Weather Context ---")
    print(f"Temperature: {weather_result.mean_temperature:.1f}°C (range: {weather_result.temperature_min:.1f} to {weather_result.temperature_max:.1f}°C)")
    print(f"Precipitation: {weather_result.total_precipitation:.1f} mm")

print("\n" + "#"*60)

## 8. Export for Deliverable

Export results to files for client presentation.

In [ ]:
# Export DataFrames
if EXPORT_CSV:
    # Vegetation data
    veg_df = veg_result.to_dataframe()
    veg_csv_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_vegetation.csv"
    veg_df.to_csv(veg_csv_path, index=False)
    print(f"✓ Saved vegetation data: {veg_csv_path}")
    
    # Change data
    change_df = change_result.to_dataframe()
    change_csv_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_change.csv"
    change_df.to_csv(change_csv_path, index=False)
    print(f"✓ Saved change data: {change_csv_path}")
    
    # Weather data (if available)
    if weather_result.confidence > 0:
        weather_df = weather_result.to_dataframe()
        weather_csv_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_weather.csv"
        weather_df.to_csv(weather_csv_path, index=False)
        print(f"✓ Saved weather data: {weather_csv_path}")

In [ ]:
# Export PNG visualizations
if EXPORT_PNG:
    # Vegetation visualization
    veg_png_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_vegetation.png"
    veg_result.to_png(veg_png_path)
    print(f"✓ Saved vegetation chart: {veg_png_path}")
    
    # Change visualization
    change_png_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_change.png"
    change_result.to_png(change_png_path)
    print(f"✓ Saved change chart: {change_png_path}")
    
    # Weather visualization (if available)
    if weather_result.confidence > 0:
        weather_png_path = output_path / f"{CLIENT_NAME.replace(' ', '_')}_weather.png"
        weather_result.to_png(weather_png_path)
        print(f"✓ Saved weather chart: {weather_png_path}")

In [ ]:
# List all exported files
print("\n" + "="*50)
print("EXPORTED FILES")
print("="*50)
for f in sorted(output_path.glob(f"{CLIENT_NAME.replace(' ', '_')}*")):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name} ({size_kb:.1f} KB)")
print("="*50)
print(f"\nAll files saved to: {output_path.absolute()}")

## 9. Data Quality Notes

Important considerations for interpreting the results.

In [ ]:
# Data quality summary
print("\n" + "="*50)
print("DATA QUALITY NOTES")
print("="*50)

# Vegetation quality
print(f"\nVegetation Analysis:")
print(f"  - Confidence: {veg_result.confidence:.0%}")
print(f"  - Satellite passes: {veg_result.observation_count}")
print(f"  - Cloud-free passes: {veg_result.cloud_free_count}")
if veg_result.warnings:
    print(f"  - Warnings:")
    for w in veg_result.warnings:
        print(f"      ⚠ {w}")

# Change detection quality
print(f"\nChange Detection:")
print(f"  - Combined confidence: {change_result.confidence:.0%}")
print(f"  - Period 1 confidence: {change_result.period_1_confidence:.0%}")
print(f"  - Period 2 confidence: {change_result.period_2_confidence:.0%}")
if change_result.warnings:
    print(f"  - Warnings:")
    for w in change_result.warnings:
        print(f"      ⚠ {w}")

# Weather quality
if weather_result.confidence > 0:
    print(f"\nWeather Data:")
    print(f"  - Confidence: {weather_result.confidence:.0%}")
    print(f"  - Data source: {weather_result.data_source}")
    print(f"  - Observations: {weather_result.observation_count} days")
    if weather_result.warnings:
        print(f"  - Warnings:")
        for w in weather_result.warnings:
            print(f"      ⚠ {w}")

print("\n" + "="*50)

---

**Report generated with SatelliteHub**

For questions about this analysis, contact the consulting team.